In [ ]:
import pandas as pd
from datetime import datetime,timedelta
import os
from prophet import Prophet


In [9]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho)

In [10]:
tbl_analitico_path = os.path.join(path_arq, 'cubo_anl', 'tbl_full_analitica.xlsx')
tbl_ = pd.read_excel(tbl_analitico_path)


In [11]:
tbl_2 = tbl_[['CHAMADO', 'DT_ABERTURA_CHAMADO', 'MODULO_CHAMADO', 'TIPO_CHAMADO', 'COMPLEXIDADE']]


In [12]:
# Pega data minima da amostra:

dt_max = tbl_2['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').max()
dt_min = (dt_max - timedelta(days=20)).strftime('%Y-%m-%d')

In [13]:
#Agrupando e contando volume de entradas por modulo, tipo e complexidade diariamente:
df_agrupado = tbl_2.groupby(['DT_ABERTURA_CHAMADO']).agg({'CHAMADO':'count'}).reset_index().sort_values(['DT_ABERTURA_CHAMADO'])

In [14]:
#Filtrando a partir da data minima:
df_agrupado = df_agrupado[df_agrupado['DT_ABERTURA_CHAMADO']>= dt_min]
df_agrupado['DT_ABERTURA_CHAMADO'] = df_agrupado['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]')

In [15]:
df_prophet = df_agrupado[['DT_ABERTURA_CHAMADO', 'CHAMADO']].rename(columns={'DT_ABERTURA_CHAMADO': 'ds', 'CHAMADO': 'y'})

In [ ]:
model = Prophet(changepoint_prior_scale=0.5, seasonality_mode='additive')
model.fit(df_prophet)

In [ ]:
dt_max.strftime('%Y-%m-%d')

In [ ]:
intervalo_datas = pd.date_range(start=dt_max, periods=30, freq='D')
proximos_30_dias = intervalo_datas[intervalo_datas.weekday < 5][:30] 

previsao_futura = pd.DataFrame({'ds': proximos_30_dias})
forecast = model.predict(previsao_futura)

In [ ]:
# Filtrando as previsões a partir de 2024-04-02
previsao_futura = forecast[forecast['ds'] > dt_max]

# Exibindo as previsões para o período desejado
previsao_futura = previsao_futura[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

#criando ponderados que faz a média entre a média mais o desvio com a média prevista
previsao_futura['previsao'] = round(previsao_futura['yhat'],0)

In [ ]:
# Define the output folder for the model forecast
output_folder_modelo = os.path.join(path_arq, 'modelo')

# Create the output folder if it doesn't exist
os.makedirs(output_folder_modelo, exist_ok=True)

# Define the output file path for the forecast
output_file_previsao = os.path.join(output_folder_modelo, 'Previsao_demandas.xlsx')

# Save the forecast DataFrame to an Excel file (output)
previsao_futura[['ds', 'previsao']].to_excel(output_file_previsao, index=False)

print(f"Forecast output saved to: {output_file_previsao}")
